## Installs and imports

In [1]:
from pyscf import gto, scf, ao2mo
import numpy as np
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit_nature.second_q.drivers import PySCFDriver
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from qiskit_aer import Aer
from qiskit_algorithms import VQE
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.primitives import Estimator
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_algorithms.optimizers import SPSA
from qiskit_nature.second_q.algorithms import QEOM, GroundStateSolver
from IPython.display import Image
from qiskit_algorithms import VQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.problems import BaseProblem
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT
from qiskit.quantum_info import SparsePauliOp
import numpy as np
from qiskit import QuantumCircuit
from qsharp.interop.qiskit import estimate

In [2]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

## Construct QPE circuit for large molecules

In [3]:
def get_fermionic_op(atom_string: str, spin: int, charge: int, basis: str = "sto-3g"):
    driver = PySCFDriver(
        atom=atom_string,
        basis=basis,
        charge=charge,
        spin=spin,
        unit=DistanceUnit.ANGSTROM,
    )
    problem = driver.run()

    # Convert to FermionicOp
    atom_hamiltonian = problem.second_q_ops()

    return atom_hamiltonian[0]

In [4]:
StringH2  = "H 0 0 0; H 0 0 0.735"
h2_mol_op = get_fermionic_op(
    atom_string = StringH2,
    spin        = 0,
    charge      = 0
)

StringLiH = "Li 0 0 0; H 0.5 0.5 0.5"
lih_mol_op = get_fermionic_op(
    atom_string = StringLiH,
    spin        = 0,
    charge      = 0
)

In [5]:
mapper       = JordanWignerMapper()
h2_qubit_op  = mapper.map(h2_mol_op)
lih_qubit_op = mapper.map(lih_mol_op)

In [12]:
def generate_controlled_time_evolution(hamiltonian: SparsePauliOp, time: float):
    n_qubits = hamiltonian.num_qubits
    ctrl     = QuantumRegister(1)
    system   = QuantumRegister(n_qubits)
    circuit  = QuantumCircuit(ctrl, system)

    for pauli_string, coeff in zip(hamiltonian.paulis, hamiltonian.coeffs):
        # For each Pauli term, we build a controlled e^{-i c P t}
        pauli_label = pauli_string.to_label()
        theta = -2 * coeff.real * time

        # Basis change to Z
        for i, p in enumerate(pauli_label):
            if p == 'X':
                circuit.h(system[i])
            elif p == 'Y':
                circuit.sdg(system[i])
                circuit.h(system[i])
            # If 'Z' or 'I', no basis change needed

        # Controlled multi-Z rotation
        qubit_indices = [i for i, p in enumerate(pauli_label) if p != 'I']
        if qubit_indices:
            if len(qubit_indices) == 1:
                target = qubit_indices[0]
                circuit.cx(ctrl[0], system[target])
                circuit.rz(theta, system[target])
                circuit.cx(ctrl[0], system[target])
            else:
                # Multi-qubit controlled rotation
                for q in qubit_indices[:-1]:
                    circuit.cx(ctrl[0], system[q])
                circuit.mcx(qubit_indices[:-1], system[qubit_indices[-1]])
                circuit.rz(theta, system[qubit_indices[-1]])
                circuit.mcx(qubit_indices[:-1], system[qubit_indices[-1]])
                for q in reversed(qubit_indices[:-1]):
                    circuit.cx(ctrl[0], system[q])

        # Undo basis change
        for i, p in enumerate(pauli_label):
            if p == 'X':
                circuit.h(system[i])
            elif p == 'Y':
                circuit.h(system[i])
                circuit.s(system[i])

    return circuit

def build_qpe_circuit(qubit_hamiltonian: SparsePauliOp, n_ancillas: int = 5, t0: float = 1.0):
    """Build the full QPE circuit for a given qubit Hamiltonian."""
    n_system = qubit_hamiltonian.num_qubits

    # Create quantum registers
    ancillas = QuantumRegister(n_ancillas)
    system = QuantumRegister(n_system)
    classical = ClassicalRegister(n_ancillas)
    qc = QuantumCircuit(ancillas, system, classical)

    # Step 1: Hadamard gates on ancillas
    qc.h(ancillas)

    # Step 2: Prepare system (for now identity, you can load HF state here)
    # Optionally prepare a Hartree-Fock state if desired

    # Step 3: Controlled evolutions
    for k in range(n_ancillas):
        t = (2 ** k) * t0
        controlled_evo = generate_controlled_time_evolution(qubit_hamiltonian, t)
        qc.compose(controlled_evo, qubits=[ancillas[k]] + system[:], inplace=True)

    # Step 4: Inverse QFT
    qc.compose(QFT(num_qubits=n_ancillas, inverse=True, do_swaps=True), ancillas, inplace=True)

    # Step 5: Measure ancillas
    qc.measure(ancillas, classical)

    return qc

def estimate_resources(quantum_circuit):
    result = estimate(quantum_circuit)
    return result['physicalCounts']['physicalQubits'], result['physicalCounts']['runtime']

In [13]:
h2_circ = build_qpe_circuit(h2_qubit_op)
h2_circ = h2_circ.decompose().decompose()

In [14]:
h2_ft_qc_qubits, h2_ft_qc_runtime = estimate_resources(h2_circ)
print(f"QPE circuit for H2 molecule:")
print(f"  Original Circuit: Qubits = {h2_circ.num_qubits}")
print(f"  Fault Tolerant Circuit: Qubits = {h2_ft_qc_qubits}, Runtime = {h2_ft_qc_runtime}")

QPE circuit for H2 molecule:
  Original Circuit: Qubits = 9
  Fault Tolerant Circuit: Qubits = 426600, Runtime = 217470000


In [15]:
lih_circ = build_qpe_circuit(lih_qubit_op)
lih_circ = lih_circ.decompose().decompose()

In [16]:
lih_ft_qc_qubits, lih_ft_qc_runtime = estimate_resources(lih_circ)
print(f"QPE circuit for LiH molecule:")
print(f"  Original Circuit: Qubits = {lih_circ.num_qubits}")
print(f"  Fault Tolerant Circuit: Qubits = {lih_ft_qc_qubits}, Runtime = {lih_ft_qc_runtime}")

QPE circuit for LiH molecule:
  Original Circuit: Qubits = 17
  Fault Tolerant Circuit: Qubits = 807814, Runtime = 184222920000
